In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from starfish.common.env_loader import load_env_file
from starfish.core.structured_llm import get_structured_llm

load_env_file()

2025-04-02 08:55:36 | INFO     | starfish.common.env_loader | env_loader.py:54 | Loaded 9 environment variables from /Users/zhengisamazing/1.python_dir/starfish/.env


True

In [3]:
first_llm = get_structured_llm(
    model_name="openai/gpt-4o-mini",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York")
first_response.data

[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, New York City has an estimated population of over 8.4 million residents.'}]

In [5]:
### Local model
first_llm = get_structured_llm(
    model_name="ollama/gemma3:1b",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York")
first_response.data

2025-04-02 00:29:20 | INFO     | starfish.adapters.litellm_adapter | litellm_adapter.py:90 | Ensuring Ollama model gemma3:1b is ready...
2025-04-02 00:29:20 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:50 | Ollama server is already running
2025-04-02 00:29:20 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:132 | Found model gemma3:1b
2025-04-02 00:29:20 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:238 | Model gemma3:1b is already available
2025-04-02 00:29:20 | INFO     | starfish.adapters.litellm_adapter | litellm_adapter.py:99 | Model gemma3:1b is ready, making API call...


[{'question': 'What are some notable facts about New York City?',
  'answer': "Here are some notable facts about New York City:\n\n*   **Largest City in the US:** New York City is the most populous city in the United States.\n*   **Cultural Hub:** It's a global center for arts, culture, and entertainment, hosting world-class museums, theaters, and music venues.\n*   **Financial Center:** It's a major financial hub, home to Wall Street and many major financial institutions.\n*   **Diverse Population:** New York City is incredibly diverse, with a significant population of immigrants from all over the world.\n*   **Iconic Landmarks:** It boasts famous landmarks like the Statue of Liberty, Times Square, Central Park, and the Empire State Building.\n*   **Major Industries:**  It’s a leader in industries like media, fashion, finance, technology, and healthcare.\n*   **Historical Significance:**  It has a long and complex history, having been a major center of trade, immigration, and revoluti

In [7]:
from starfish.adapters.ollama_adapter import stop_ollama_server

await stop_ollama_server()

2025-04-02 00:29:51 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:259 | Stopping Ollama server...
2025-04-02 00:29:52 | INFO     | starfish.adapters.ollama_adapter | ollama_adapter.py:309 | Ollama server stopped successfully


True